This notebook is designed to quickly demo major capabilities of the note summarization module without having to deploy and configure all the relevant components of the solution. The notebook:
1. Installs all necessary dependencies.
1. Creates a SQLite db file healthcare_data.db and ingests sample patient data from .csv files (located in a subfolder data) in the db.
2. Runs sample prompts for a specific patient to show case how [functional requirements](ref_design.md).

To make run the notebook refer to additional information below.

1. **Install Dependencies**:
    - The required dependencies are installed using a `requirements.txt` file.

In [13]:
#%pip install -r requirements.txt
#%conda install --file requirements.txt

2. **Import CSV Files into SQLite Database**:
    - The CSV files are read from a specified directory and imported into a SQLite database named `healthcare_data.db`.
    - Each CSV file is imported into a corresponding table in the database.

In [1]:
import sqlite3
import pandas as pd

# Create a SQLite database
conn = sqlite3.connect('healthcare_data.db')

# List of CSV files to import
csv_files = [
    'allergies.csv', 'careplans.csv', 'claims.csv', 'claims_transactions.csv', 'conditions.csv',
    'devices.csv', 'encounters.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv',
    'observations.csv', 'organizations.csv', 'patients.csv', 'payer_transitions.csv', 'payers.csv',
    'procedures.csv', 'providers.csv', 'supplies.csv'
]

DATA_DIR = "./data"  # Directory with your CSVs

# Import each CSV file into a table in the SQLite database
for file in csv_files:
    table_name = file.split('.')[0]
    file = f'{DATA_DIR}/{file}'
    df = pd.read_csv(file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()


3. **Run Sample Prompts**:
    - The necessary modules (`sqlite3`, `SQLDatabase`, `ChatOpenAI`, `SQLDatabaseChain`) are imported.
    - The OpenAI API key is set up using an environment variable.
    - Templates for generating SQL queries and prompts for different patient data categories (e.g., medications, immunizations) are defined.
    - A connection to the SQLite database is established.
    - The LangChain `SQLDatabase` and OpenAI model (`ChatOpenAI`) are initialized.
    - A `SQLDatabaseChain` is created to facilitate SQL query generation from natural language prompts.
    - `generate_sql_query(template, first_name, last_name)`: Generates SQL queries based on the provided template and patient name.
    - `execute_query(query)`: Executes the generated SQL query on the SQLite database and fetches the results.
    - `format_data(template, data)`: Formats the extracted data into a specified template.
    - `get_summary_from_openai(prompt)`: Sends a prompt to the OpenAI model and retrieves the response.
    - `generate_patient_summary(first_name, last_name)`: Generates a summary for a patient using all defined templates. It involves generating SQL queries, executing them, formatting the data, and obtaining a summary from the OpenAI model.
    - An example usage of the `generate_patient_summary` function is provided for a patient named "Lupe126 Rippin620".

In [3]:
import os
import getpass
import json
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
#from langchain_openai import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts.prompt import PromptTemplate

if not os.getenv("OPENAI_API_KEY"):
    API_KEY = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = API_KEY # Replace with your OpenAI API key

# Constants
system_prompt = "Patient {first_name} {last_name}. "

patient_templates = {
    "medications": {
        "sql_prompt": "What medications are prescribed to the patient named {first_name} {last_name}? Retrieve ALL medications for the patient.", #Do NOT explain how you are going to do that, just return the sql query. Do NOT split the query into multiple lines.",
        "entities": [ 
            {
                "name":  "medications",
                "fields": ["start", "description", "reasondescription"]
            }
        ],
        "prompt": "Summarize the patient's current medications:\n {medications_rows}"
    },
    # "symptoms": {
    #     "sql_prompt": "In a single query retrieve ALL encounters of the patient named {first_name} {last_name} and for each encounter relevant conditions and observations.", #Do NOT explain how you are going to do that, just return the sql query. Do NOT split the query into multiple lines.",
    #     "entities": [ 
    #         {
    #             "name":  "encounters",
    #             # the generated query changes the column names, so they are used below instead.
    #             "fields": ["encounter_start", "encounter_stop","encounter_description", "condition_code", "condition_description", "observation_description", "observation_value", "observation_units"]
    #         }
    #     ],
    #     "prompt": "Provide an overview of the patient's reported symptoms and their progression over time:\n {encounters_rows}\n"
    # },
    # "immunizations": {
    #     "sql_prompt" : "Retrieve ALL immunizations for the patient named {first_name} {last_name}.", 
    #     "entities": [ 
    #         {
    #             "name":  "immunizations",
    #             "fields": ["date", "description"]
    #         }
    #     ],
    #     "prompt": "Summarize the patient's immunizations:\n {immunizations_rows}"
    # }
}
#    "physical_exam": "What are the key points from the patient's last physical examination?",
#    "allergies": "Highlight any noted allergies or adverse reactions documented in the patient's records.",
#    "consultation": "What are the key findings from the patient's last consultation note?",
#    "history": "Summarize the patient's medical history relevant to their current condition.",
#    "chronic_conditions": "Highlight any chronic conditions and their management plans documented in the patient's history.",


# Connect to the SQLite database (or another DB)
db_path = "./healthcare_data.db"
conn = sqlite3.connect(db_path)

# Initialize LangChain SQLDatabase
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Initialize OpenAI model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Create a database chain
# By default the chain will not only generate SQL queries but also execute them on the database. In case of queries that contain the JOIN statement.
# the chain will fail to execute them. To avoid this, set the return_sql parameter to True.
# FIXME:
# The new prompt template should also instruct the LLM to return the response in structured output format to reduce the chances of halucinations.

# By default the chain uses a default _sqlite_prompt (refer to site-packages\langchain\chains\sql_database\prompt.py) which imposes a requirement on the format that the LLM should return its response in.
# Create a custom prompt template
sqlite_prompt = """You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. 
The database schema consists of multiple tables, each containing different columns.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. 
Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables provided. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Return only the SQL query as the answer. Do not include any explanations, formatting, such as ```sql,  or additional text.

Only use the following tables: {table_info}

Question: {input}"""

prompt = PromptTemplate(
    input_variables=["input", "table_info"],
    template=sqlite_prompt
)

# Initialize the database chain with the custom prompt
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=prompt, return_sql=True)

def generate_sql_query(template, first_name, last_name):
    """Generate SQL query, ensuring it is patient-specific."""
    full_sql_prompt = system_prompt.format(first_name=first_name, last_name=last_name) + template["sql_prompt"].format(first_name=first_name, last_name=last_name)

    # Generate SQL from text query
    response = db_chain.invoke(full_sql_prompt)
    sql_query = response["result"]
 
    # print(f"Generated SQL query: {sql_query}")  # Debugging step

    return sql_query

def execute_query(query):
    """Execute SQL query on SQLite database and fetch results"""
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows

def format_data(template, data):
    """Format extracted data into the template."""
    formatted_rows = "\n".join([", ".join(map(str, row)) for row in data])
    entity = template["entities"][0]
    entity_name = entity["name"]
    entity_fields = ", ".join(entity["fields"])
    return template["prompt"].replace(f"{{{entity_name}_rows}}", formatted_rows)
  

def get_summary_from_openai(prompt):
    """Send prompt to OpenAI model and get a response."""
    response = llm.client.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def generate_patient_summary(first_name, last_name):
    """Generate a patient summary using all templates."""
    print(f"Generating summary for {first_name} {last_name}\n")

    for key, template in patient_templates.items():
        print("=====================================")
        print(f"{key}")
        print("=====================================")
        query = generate_sql_query(template, first_name, last_name)
        data = execute_query(query)
        formatted_prompt = format_data(template, data)
        summary = get_summary_from_openai(formatted_prompt)
        print(f"\n{summary}\n")

    conn.close()

# Example usage
generate_patient_summary("Lupe126", "Rippin620")


C:\Users\Olga\AppData\Local\Temp\ipykernel_3744\1218041501.py:65: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


Generating summary for Lupe126 Rippin620

medications

The patient is currently taking the following medications:

1. **Loratadine 5 MG Chewable Tablet**: Used as an antihistamine for allergy symptoms. No end date provided, suggesting it may be used as needed or ongoing.

2. **Epinephrine 1 MG/ML Auto-Injector (NDA020800 0.3 ML)**: Typically used for emergency treatment of severe allergic reactions (anaphylaxis). No end date provided, indicating it is on hand for emergencies.

Past medications include:

1. **Ibuprofen 100 MG Oral Tablet**: Used for pain relief or anti-inflammatory purposes. The course ended on 2015-12-24.

2. **Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet**: An antibiotic used for bacterial infections, which was prescribed for a condition related to viral sinusitis, but was discontinued as of 2016-08-29.

Keep in mind that this information is based on provided data entries and might not reflect all current prescriptions or over-the-counter medications the patien